In [1]:
import tensorflow as tf

In [2]:
# for any evaluation of a function, we need to open a tensorflow session
#and use to initialze the variables and evaluate the function.
x = tf.Variable(3, name = "X")
y = tf.Variable(4, name = "y")
f = x*x*y + y + 2

sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print (result)

42


In [3]:
# the better way
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    
    
# instead of manually running the initializer for every single variable
# you can use the global_variable_initializer() function.

init = tf.global_variables_initializer() # prepare an init node
with tf.Session() as sess:
    init.run() # actually initialize all the variables
    result = f.eval()

In [4]:
# creating interactive session
# the only difference between interactive session and regular session
# is that when an interactive session is created it automatically sets
# itself as the default session so the with block is not needed to close it

sess = tf.InteractiveSession()
init.run()
result = f.eval()
print (result)
sess.close()

42


In [5]:
# managing graphs
# any node created is automatically added to the default graph

x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [6]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    
x2.graph is graph

True

In [7]:
x2.graph is tf.get_default_graph()

False

In [8]:
w = tf.constant(3)
x = w+ 2
y = x + 5
z = x* 3

with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print (y_val)
    print (z_val)

10
15


In [9]:
# linear regression with tensorflow
import sklearn as sk

import numpy as np
from sklearn.datasets import load_iris
iris_data = load_iris()

In [10]:
x = iris_data.data
y = iris_data.target
x.shape

(150, 4)

In [11]:
m, n = x.shape
iris_data_plus_bias = np.c_[np.ones((m, 1)), x]

In [12]:
X = tf.constant(iris_data_plus_bias, dtype = tf.float32, name = "X")
Y = tf.constant(y.reshape(-1,1), dtype = tf.float32,name = "y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), Y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [13]:
# implementing gradient descent

n_epoachs = 1000
learning_rate = 0.01
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()
scaled_iris_data_plus_bias = scale.fit_transform(iris_data_plus_bias)

X = tf.constant(scaled_iris_data_plus_bias, dtype = tf.float32, name = 'X')
Y = tf.constant(y.reshape(-1,1), dtype = tf.float32, name = 'Y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0,1.0), name = 'theta')

y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - Y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)

training_op = tf.assign(theta, theta- learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epoachs):
        if epoch % 100 == 0:
            print ("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()


Epoch 0 MSE =  3.98892
Epoch 100 MSE =  0.187382
Epoch 200 MSE =  0.097204
Epoch 300 MSE =  0.0854662
Epoch 400 MSE =  0.0782897
Epoch 500 MSE =  0.0732414
Epoch 600 MSE =  0.0696228
Epoch 700 MSE =  0.0669795
Epoch 800 MSE =  0.0650042
Epoch 900 MSE =  0.0634883


In [14]:
# using tensorflow autodiff feature to implement gradient descent

# implementing gradient descent

n_epoachs = 1000
learning_rate = 0.01
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()
scaled_iris_data_plus_bias = scale.fit_transform(iris_data_plus_bias)

X = tf.constant(scaled_iris_data_plus_bias, dtype = tf.float32, name = 'X')
Y = tf.constant(y.reshape(-1,1), dtype = tf.float32, name = 'Y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0,1.0), name = 'theta')

y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - Y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
gradients = tf.gradients(mse, [theta])[0]

training_op = tf.assign(theta, theta- learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epoachs):
        if epoch % 100 == 0:
            print ("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()


Epoch 0 MSE =  2.14705
Epoch 100 MSE =  0.214926
Epoch 200 MSE =  0.136261
Epoch 300 MSE =  0.107502
Epoch 400 MSE =  0.0884958
Epoch 500 MSE =  0.0756143
Epoch 600 MSE =  0.0668703
Epoch 700 MSE =  0.060928
Epoch 800 MSE =  0.0568832
Epoch 900 MSE =  0.0541239


In [15]:
# using optimizers implemented in tensorflow
# that is the gradient descent optimizer

# implementing gradient descent

n_epoachs = 1000
learning_rate = 0.01
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()
scaled_iris_data_plus_bias = scale.fit_transform(iris_data_plus_bias)

X = tf.constant(scaled_iris_data_plus_bias, dtype = tf.float32, name = 'X')
Y = tf.constant(y.reshape(-1,1), dtype = tf.float32, name = 'Y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0,1.0), name = 'theta')

y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - Y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)

training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epoachs):
        if epoch % 100 == 0:
            print ("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()


Epoch 0 MSE =  2.4262
Epoch 100 MSE =  0.23988
Epoch 200 MSE =  0.152016
Epoch 300 MSE =  0.120196
Epoch 400 MSE =  0.0991039
Epoch 500 MSE =  0.0847354
Epoch 600 MSE =  0.0749116
Epoch 700 MSE =  0.068168
Epoch 800 MSE =  0.0635132
Epoch 900 MSE =  0.0602761


In [16]:
# feeding data to the training algorithm

# for this we need a way to replace x and y at every iteration witht eh
# next-batch. The simplest way to do this is to use placeholder nodes.

# to create a placeholder node, you must call the placeholder() function
# and specify the ouput tensor's data type. If you specify none for
# a dimension, it means "any sizer".

A = tf.placeholder(tf.float32, shape = (None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict = {A:[[1,2,3]]})
    B_val_2 = B.eval(feed_dict = {A:[[4,5,6], [7,8,9]]})
    
print (B_val_1)
print (B_val_2)

[[ 6.  7.  8.]]
[[  9.  10.  11.]
 [ 12.  13.  14.]]


In [28]:
# to implement Mini-batch Gradient Descent, we only need to tweek existing code
#slightly. First change the definiation of x and y in the construcion
# phase to make them placeholder nodes:

x = tf.placeholder(tf.float32, shape = (None, n), name = "X")
y = tf.placeholder(tf.float32, shape = (150, ), name = "y")

# then define the batch size and compute the total number of batches
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

In [29]:
from sklearn.datasets import load_iris
iris_data = load_iris()
x_batch, y_batch = iris_data.data, iris_data.target
x_batch.shape, y_batch.shape

((150, 4), (150,))

In [30]:
# finally, in the execution pahse, fetch the mini-batches one by one,
# then provice the value of x and y via the feed_dict parameter when evaluating
#a node that depends on either of them.

def fetch_batch(epoch, batch_index, batch_size):
    from sklearn.datasets import load_iris
    iris_data = load_iris()
    x_batch, y_batch = iris_data.data, iris_data.target
    return x_batch, y_batch

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epoachs):
        for batch_index in range(n_batches):
            a_batch, b_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict = {x: a_batch, y:b_batch})
            best_theta = theta.eval()

In [32]:
# saving and restoring models

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epoachs):
        if epoch % 100 == 0: # checkpoint every 100 epochs
            save_path = saver.save(sess, r"C:\Users\ACER\Desktop\pyreach\Life Data\my_model.ckpt")
 
        sess.run(training_op)

    best_theta = theta.eval()
    save_path = saver.save(sess, r"C:\Users\ACER\Desktop\pyreach\Life Data\my_model.ckpt")

In [ ]:
# to restore a model
    
# we can also specify what variable to be restored or saved
# e.g

saver = tf.train.Saver({"weights":theta})
# this implies we save or restore the theta variable under the name weights

# by default the save() method also saves the struture of the graph in a seconde
# file with the same name plus a .meta extension. you can load this graph
# structure using tf.train.import_meta_graph().

saver = tf.train.import_meta_graph(r"C:\Users\ACER\Desktop\pyreach\Life Data\my_model.ckpt.meta")

with tf.Session() as sess:
    saver.restore(sess, r"C:\Users\ACER\Desktop\pyreach\Life Data\my_model.ckpt.meta")

In [ ]:
# visualizing the graph and training curves using TensorBoard

from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = 'tf_logs'
logdir = "{}/run-{}/".format(root_logdir, now)

mse_summary= tf.summary.scaler("MSE",mse) # creates node in the graph
# that will evaluate the MSE value and write it to a tensorBoard-compatible
# binary log string called a summary.

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())


In [ ]:
#  to updatet the execution phase to evaluate the mse_summary node regularly
# during trianing (e.g. every 10 mini-batches)

for batch_index in range(n_batches):
    x_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
    if batch_index % 10 == 0:
        summary_str = mse_summary.eval(feed_dict = {X:x_batch, y:y_batch})
        step = epoch * n_batches + batch_index
        file_writer.add_summary(summary_str, step)
    sess.run(training_op, feed_dict = {X:x_batch, y:y_batch})
    
# finally closing the filewriter at the end of the file
file_writer.close()